# Creating customer status New, Existing, Retained, Churned, Reactivated and Dormant

The goal here is to create the various customer status columns we need to properly analyse our customers. We will create the status "Retained", "Churned", "Reactivated", "Dormant", "Existing" and "Non Existing" based on a sales order csv table that contains only order_id, customer_id and order_date. We will use one month as the period.

Usually we would code directly the SQL to extract the data from a server but in this case we are running everything from a jupyter notebook so I will add the additional python code part where we get the data from a CSV file, we create a database, and then we use SQLite within the jupyter notebook.

To get directly to the SQL code, skip this part and scroll down to "Beginning of SQL Part".

#### Import the sales order data and check

In [1]:
import pandas as pd

In [2]:
df_orders = pd.read_csv("orders.csv")

In [3]:
df_orders.shape

(77280, 10)

In [4]:
df_orders.head(2)

,order_id,order_status_id,customer_id,shipping_method_id,order_date,country_id,snapshot_customer_email_status_id,promo_code_id,marketing_channel_id,site_id
0,1,7,12719,(null),2008-07-21,89,1,(null),(null),1
1,2,7,12943,(null),2008-11-03,220,1,(null),(null),1


In [5]:
# The table has extra columns that we don't want. We want to create all the status using only order_id, customer_id, order_date
# The columns we want are : order_id, customer_id, order_date
df_orders_2 = df_orders[['order_id', 'customer_id', 'order_date']]

# We check
df_orders_2.head(2)

,order_id,customer_id,order_date
0,1,12719,2008-07-21
1,2,12943,2008-11-03


#### Import a table containing all the months from 2008 to 2020

This months table is necessary to have rows of data where there were no transactions in order to identify the churned month later on.

In [6]:
df_months = pd.read_csv("months.csv")

In [7]:
df_months.shape

(156, 1)

In [8]:
df_months.head(3)

,months
0,2008-01-01
1,2008-02-01
2,2008-03-01


In [9]:
df_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 1 columns):
months    156 non-null object
dtypes: object(1)
memory usage: 1.3+ KB


We need to change the format to date instead of object

In [10]:
df_months['months'] = pd.to_datetime(df_months['months'])

In [11]:
df_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 1 columns):
months    156 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.3 KB


#### Create the database to use SQL code

In [2]:
import sqlite3
from sqlite3 import Error

In [13]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection(r"customer_database.db")

2.6.0


In [3]:
conn = sqlite3.connect("customer_database.db")
cursor = conn.cursor()

In [15]:
# Create an empty table called orders in our new database
df_orders_2.to_sql('orders', conn, if_exists='replace', index=False)
conn.commit()

In [16]:
# Create a table called months in our new database
df_months.to_sql('months', conn, if_exists='replace', index=False)
conn.commit()

In [4]:
# We just check the tables in the database quickly 
query = 'SELECT name from sqlite_master where type = "table"'
schema = cursor.execute(query).fetchall()
for i in schema :
    print(i)

('orders',)
('months',)
('calendar_orders',)


In [5]:
# We just check the columns and format quickly of table orders
query = "pragma table_info(orders);"
schema = cursor.execute(query).fetchall()
for i in schema :
    print(i)

(0, 'order_id', 'INTEGER', 0, None, 0)
(1, 'customer_id', 'INTEGER', 0, None, 0)
(2, 'order_date', 'TIMESTAMP', 0, None, 0)


In [6]:
# We just check the columns and format quickly of table months
query = "pragma table_info(months);"
schema = cursor.execute(query).fetchall()
for i in schema :
    print(i)

(0, 'months', 'TIMESTAMP', 0, None, 0)


In [7]:
# Test that the data is correct for table orders
pd.read_sql('SELECT * FROM orders LIMIT 3', conn)

,order_id,customer_id,order_date
0,1,12719,2008-07-21 00:00:00
1,2,12943,2008-11-03 00:00:00
2,3,13150,2008-12-10 00:00:00


In [8]:
# Test that the data is correct for table months
pd.read_sql('SELECT * FROM months LIMIT 3', conn)

,months
0,2008-01-01 00:00:00
1,2008-02-01 00:00:00
2,2008-03-01 00:00:00


# Beginning of SQL Part

#### We create the query to create first_order date (registration date) column

In [9]:
%%capture
%load_ext sql
%sql sqlite:///customer_database.db

In [10]:
%%sql
-- 'We just test that data is pulling through from orders'
SELECT * 
FROM orders LIMIT 3

 * sqlite:///customer_database.db
Done.


order_id,customer_id,order_date
1,12719,2008-07-21 00:00:00
2,12943,2008-11-03 00:00:00
3,13150,2008-12-10 00:00:00


In [11]:
%%sql
WITH reg_dates AS
    (
    SELECT
        customer_id,
        MIN(order_date) AS reg_date
    FROM orders
    GROUP BY customer_id
    )

SELECT
o.order_id,
o.customer_id,
o.order_date,
r.reg_date,
DATETIME(r.reg_date, 'start of month') AS reg_month,
CASE 
    WHEN o.order_date = r.reg_date
    THEN 'new'
    ELSE 'existing'
    END AS cust_type,
'active' AS activity
FROM orders o 
LEFT JOIN reg_dates r ON o.customer_id = r.customer_id
LIMIT 25

 * sqlite:///customer_database.db
Done.


order_id,customer_id,order_date,reg_date,reg_month,cust_type,activity
1,12719,2008-07-21 00:00:00,2008-07-21 00:00:00,2008-07-01 00:00:00,new,active
2,12943,2008-11-03 00:00:00,2008-11-03 00:00:00,2008-11-01 00:00:00,new,active
3,13150,2008-12-10 00:00:00,2008-12-10 00:00:00,2008-12-01 00:00:00,new,active
4,13278,2009-01-03 00:00:00,2009-01-03 00:00:00,2009-01-01 00:00:00,new,active
5,13347,2009-01-26 00:00:00,2009-01-26 00:00:00,2009-01-01 00:00:00,new,active
6,13405,2009-02-05 00:00:00,2009-02-05 00:00:00,2009-02-01 00:00:00,new,active
7,13418,2009-02-06 00:00:00,2009-02-06 00:00:00,2009-02-01 00:00:00,new,active
8,13442,2009-02-12 00:00:00,2009-02-12 00:00:00,2009-02-01 00:00:00,new,active
9,13495,2009-02-25 00:00:00,2009-02-25 00:00:00,2009-02-01 00:00:00,new,active
10,13664,2009-03-30 00:00:00,2009-03-30 00:00:00,2009-03-01 00:00:00,new,active


By looking at row 23 and 25, we can see that our query to make "first_order" date worked. We have the date of the order of customer 12943, and the date of his first order. So we can easily now classify this order as a returning customer.

#### We can save the queried data in a new csv file

In [36]:
query_save = """
            WITH reg_dates AS
            (
            SELECT
                customer_id,
                MIN(order_date) AS reg_date
            FROM orders
            GROUP BY customer_id
            )

            SELECT
            o.order_id,
            o.customer_id,
            o.order_date,
            r.reg_date,
            DATETIME(r.reg_date, 'start of month') AS reg_month,
            CASE 
                WHEN o.order_date = r.reg_date
                THEN 'new'
                ELSE 'existing'
                END AS cust_type
            FROM orders o 
            LEFT JOIN reg_dates r ON o.customer_id = r.customer_id
            """
    
df_save_orders = pd.read_sql(query_save, conn)
df_save_orders.to_csv("orders.csv")

#### Continue the query to add Retained customers, Churned and more.

Let's now create a new table that contains all possible months for all customers since 2008 (with a cross join). We have 12 * 12 = 144 rows for our months table. And about 40'000 unique customer IDs in this case. So we will get about 50mio rows for this new table. 

#### Creating query calendar_orders (all months for all customers)

In [12]:
%%sql

WITH unique_customers AS
    (
    SELECT DISTINCT(customer_id) FROM orders
    )
    
SELECT customer_id, months
FROM   unique_customers
       CROSS JOIN months
LIMIT 3

 * sqlite:///customer_database.db
Done.


customer_id,months
12719,2008-01-01 00:00:00
12719,2008-02-01 00:00:00
12719,2008-03-01 00:00:00


#### Saving calender_orders as a new table in the database

In [30]:
query_save = """
            WITH unique_customers AS
            (
            SELECT DISTINCT(customer_id) FROM orders
            )

            SELECT customer_id, months
            FROM   unique_customers
            CROSS JOIN months
            """
# Save the query in a dataframe
df_calendar_orders = pd.read_sql(query_save, conn)

# Save the dataframe in a table called calendar_orders
df_calendar_orders.to_sql('calendar_orders', conn, if_exists='replace', index=False)
conn.commit()

In [13]:
# We just check the tables in the database quickly 
query = 'SELECT name from sqlite_master where type = "table"'
schema = cursor.execute(query).fetchall()
for i in schema :
    print(i)

('orders',)
('months',)
('calendar_orders',)


In [14]:
# We just check the columns and format quickly of table calendar_orders
query = "pragma table_info(calendar_orders);"
schema = cursor.execute(query).fetchall()
for i in schema :
    print(i)

(0, 'customer_id', 'INTEGER', 0, None, 0)
(1, 'months', 'TEXT', 0, None, 0)


In [15]:
# Test that the data is correct for table calendar_orders
pd.read_sql('SELECT * FROM calendar_orders LIMIT 3', conn)

,customer_id,months
0,12719,2008-01-01 00:00:00
1,12719,2008-02-01 00:00:00
2,12719,2008-03-01 00:00:00


#### Joining orders to calendar_orders and adding columns previous_order and cust_status

We join orders to calendar_orders, and then we create the column cust_status that contains information such as Retained, Reactivated, Dormant, Churned. Note : The LIMIT 300 I add is just to make the code run fast and check easily

In [29]:
%%sql

WITH calendar_orders_2 AS
    (
    SELECT
    c.customer_id,
    DATE(c.months, 'start of month') AS months,
    o.order_id,
    o.order_date
    FROM calendar_orders c
    LEFT JOIN orders o
    ON c.customer_id = o.customer_id AND DATE(c.months, 'start of month') = DATE(o.order_date, 'start of month')
    LIMIT 300
    ),
    
    reg_dates AS
    (
    SELECT
        customer_id,
        MIN(order_date) AS reg_date
    FROM orders
    GROUP BY customer_id
    LIMIT 300
    ),
    
    dates AS
    (
    SELECT
    c2.customer_id,
    c2.months,
    c2.order_id,
    c2.order_date,
    r.reg_date,
    DATE(r.reg_date, 'start of month') AS reg_month,
    LAG(c2.order_date) OVER (PARTITION BY c2.customer_id ORDER BY c2.months) AS prev_order_date,
    DATE(LAG(c2.order_date) OVER (PARTITION BY c2.customer_id ORDER BY c2.months), 'start of month') AS prev_order_month,
    CASE 
        WHEN c2.order_date = r.reg_date THEN 'New'
        WHEN c2.months < r.reg_date THEN 'Not Existing' ELSE 'Existing' END AS cust_type
    FROM calendar_orders_2 c2 
    LEFT JOIN reg_dates r ON c2.customer_id = r.customer_id
    ORDER BY c2.customer_id, c2.months, c2.order_id
    LIMIT 300
    ),
    
    tmpm AS
    (
    SELECT
    customer_id,
    months,
    order_id,
    order_date,
    reg_date,
    reg_month,
    prev_order_date,
    prev_order_month,
    cust_type,
    CASE
        WHEN order_date NOT NULL THEN 'Yes' ELSE 'No' END AS TM,
    CASE 
        WHEN prev_order_month = DATE(months, '-1 month') OR prev_order_month = months THEN 'Yes' ELSE 'No' END AS PM
    FROM 
    dates
    LIMIT 300
    ),
    
    status AS
    (
    SELECT
    customer_id,
    months,
    order_id,
    order_date,
    strftime('%Y', order_date) AS year,
    reg_date,
    reg_month,
    prev_order_date,
    prev_order_month,
    cust_type,
    TM,
    PM,
    CASE 
        WHEN cust_type = 'Not Existing' THEN 'Not Existing'
        WHEN cust_type = 'New' THEN 'New' 
        WHEN cust_type = 'Existing' AND TM = 'No' THEN 'Non Active'
        WHEN cust_type = 'Existing' AND TM = 'Yes' THEN 'Active'
        ELSE 'Check' END AS cust_base,
    CASE 
        WHEN cust_type = 'Not Existing' THEN 'Not Existing'
        WHEN cust_type = 'New' THEN 'New'
        WHEN TM = 'Yes' AND PM = 'No' THEN 'Reactivated' 
        WHEN TM = 'Yes' AND PM = 'Yes' THEN 'Retained'
        WHEN TM = 'No' AND PM = 'No' THEN 'Dormant'
        WHEN TM = 'No' AND PM = 'Yes' THEN 'Churned'
        ELSE 'Check' END AS cust_status
    FROM tmpm
    )
    
SELECT * FROM status 
WHERE customer_id = 12943
LIMIT 30


 * sqlite:///customer_database.db
Done.


customer_id,months,order_id,order_date,year,reg_date,reg_month,prev_order_date,prev_order_month,cust_type,TM,PM,cust_base,cust_status
12943,2008-01-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-02-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-03-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-04-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-05-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-06-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-07-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-08-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-09-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing
12943,2008-10-01,None,None,None,2008-11-03 00:00:00,2008-11-01,None,None,Not Existing,No,No,Not Existing,Not Existing


#### We make this lighter by removing Dormant customers

The above is nice to have, but it's heavy, because we are not using a proper server and there's over 5mio rows. If we had direct access to the server we could keep this as is. To do this, we filter out status 'dormant' and type 'not existing'. I also add a LIMIT 2000 just to make the code run fast and to check easily.

In [40]:
%%sql

WITH calendar_orders_2 AS
    (
    SELECT
    c.customer_id,
    DATE(c.months, 'start of month') AS months,
    o.order_id,
    o.order_date
    FROM calendar_orders c
    LEFT JOIN orders o
    ON c.customer_id = o.customer_id AND DATE(c.months, 'start of month') = DATE(o.order_date, 'start of month')
    LIMIT 2000
    ),
    
    reg_dates AS
    (
    SELECT
        customer_id,
        MIN(order_date) AS reg_date
    FROM orders
    GROUP BY customer_id
    LIMIT 1000
    ),
    
    dates AS
    (
    SELECT
    c2.customer_id,
    c2.months,
    c2.order_id,
    c2.order_date,
    r.reg_date,
    DATE(r.reg_date, 'start of month') AS reg_month,
    LAG(c2.order_date) OVER (PARTITION BY c2.customer_id ORDER BY c2.months) AS prev_order_date,
    DATE(LAG(c2.order_date) OVER (PARTITION BY c2.customer_id ORDER BY c2.months), 'start of month') AS prev_order_month,
    CASE 
        WHEN c2.order_date = r.reg_date THEN 'New'
        WHEN c2.months < r.reg_date THEN 'Not Existing' ELSE 'Existing' END AS cust_type
    FROM calendar_orders_2 c2 
    LEFT JOIN reg_dates r ON c2.customer_id = r.customer_id
    ORDER BY c2.customer_id, c2.months, c2.order_id
    LIMIT 2000
    ),
    
    tmpm AS
    (
    SELECT
    customer_id,
    months,
    order_id,
    order_date,
    reg_date,
    reg_month,
    prev_order_date,
    prev_order_month,
    cust_type,
    CASE
        WHEN order_date NOT NULL THEN 'Yes' ELSE 'No' END AS TM,
    CASE 
        WHEN prev_order_month = DATE(months, '-1 month') OR prev_order_month = months THEN 'Yes' ELSE 'No' END AS PM
    FROM 
    dates
    LIMIT 2000
    ),
    
    status AS
    (
    SELECT
    customer_id,
    months,
    order_id,
    order_date,
    strftime('%Y', order_date) AS year,
    reg_date,
    reg_month,
    prev_order_date,
    prev_order_month,
    cust_type,
    TM,
    PM,
    CASE 
        WHEN cust_type = 'Not Existing' THEN 'Not Existing'
        WHEN cust_type = 'New' THEN 'New' 
        WHEN cust_type = 'Existing' AND TM = 'No' THEN 'Non Active'
        WHEN cust_type = 'Existing' AND TM = 'Yes' THEN 'Active'
        ELSE 'Check' END AS cust_base,
    CASE 
        WHEN cust_type = 'Not Existing' THEN 'Not Existing'
        WHEN cust_type = 'New' THEN 'New'
        WHEN TM = 'Yes' AND PM = 'No' THEN 'Reactivated' 
        WHEN TM = 'Yes' AND PM = 'Yes' THEN 'Retained'
        WHEN TM = 'No' AND PM = 'No' THEN 'Dormant'
        WHEN TM = 'No' AND PM = 'Yes' THEN 'Churned'
        ELSE 'Check' END AS cust_status
    FROM tmpm
    )
    
SELECT * FROM status 
WHERE cust_status <> 'Dormant' AND cust_type <> 'Not Existing'
ORDER BY months, order_date, order_id
LIMIT 40

 * sqlite:///customer_database.db
Done.


customer_id,months,order_id,order_date,year,reg_date,reg_month,prev_order_date,prev_order_month,cust_type,TM,PM,cust_base,cust_status
12719,2008-07-01,1,2008-07-21 00:00:00,2008,2008-07-21 00:00:00,2008-07-01,None,None,New,Yes,No,New,New
12719,2008-08-01,None,None,None,2008-07-21 00:00:00,2008-07-01,2008-07-21 00:00:00,2008-07-01,Existing,No,Yes,Non Active,Churned
12943,2008-11-01,2,2008-11-03 00:00:00,2008,2008-11-03 00:00:00,2008-11-01,None,None,New,Yes,No,New,New
12943,2008-12-01,None,None,None,2008-11-03 00:00:00,2008-11-01,2008-11-03 00:00:00,2008-11-01,Existing,No,Yes,Non Active,Churned
13150,2008-12-01,3,2008-12-10 00:00:00,2008,2008-12-10 00:00:00,2008-12-01,None,None,New,Yes,No,New,New
13150,2009-01-01,None,None,None,2008-12-10 00:00:00,2008-12-01,2008-12-10 00:00:00,2008-12-01,Existing,No,Yes,Non Active,Churned
13278,2009-01-01,4,2009-01-03 00:00:00,2009,2009-01-03 00:00:00,2009-01-01,None,None,New,Yes,No,New,New
13347,2009-01-01,5,2009-01-26 00:00:00,2009,2009-01-26 00:00:00,2009-01-01,None,None,New,Yes,No,New,New
13278,2009-02-01,None,None,None,2009-01-03 00:00:00,2009-01-01,2009-01-03 00:00:00,2009-01-01,Existing,No,Yes,Non Active,Churned
13347,2009-02-01,None,None,None,2009-01-26 00:00:00,2009-01-01,2009-01-26 00:00:00,2009-01-01,Existing,No,Yes,Non Active,Churned


#### Conclusion

We now have a table that contains information on our customers such as new, existing, retained, reactivated and churned, for all the orders of all customers. We could also have dormant but we removed it to make it lighter.

#### We can export this new table to csv

In [41]:
query_orders_cust_status = """
                WITH calendar_orders_2 AS
                    (
                    SELECT
                    c.customer_id,
                    DATE(c.months, 'start of month') AS months,
                    o.order_id,
                    o.order_date
                    FROM calendar_orders c
                    LEFT JOIN orders o
                    ON c.customer_id = o.customer_id AND DATE(c.months, 'start of month') = DATE(o.order_date, 'start of month')
                    ),

                    reg_dates AS
                    (
                    SELECT
                        customer_id,
                        MIN(order_date) AS reg_date
                    FROM orders
                    GROUP BY customer_id
                    ),

                    dates AS
                    (
                    SELECT
                    c2.customer_id,
                    c2.months,
                    c2.order_id,
                    c2.order_date,
                    r.reg_date,
                    DATE(r.reg_date, 'start of month') AS reg_month,
                    LAG(c2.order_date) OVER (PARTITION BY c2.customer_id ORDER BY c2.months) AS prev_order_date,
                    DATE(LAG(c2.order_date) OVER (PARTITION BY c2.customer_id ORDER BY c2.months), 'start of month') AS prev_order_month,
                    CASE 
                        WHEN c2.order_date = r.reg_date THEN 'New'
                        WHEN c2.months < r.reg_date THEN 'Not Existing' ELSE 'Existing' END AS cust_type
                    FROM calendar_orders_2 c2 
                    LEFT JOIN reg_dates r ON c2.customer_id = r.customer_id
                    ORDER BY c2.customer_id, c2.months, c2.order_id
                    ),

                    tmpm AS
                    (
                    SELECT
                    customer_id,
                    months,
                    order_id,
                    order_date,
                    reg_date,
                    reg_month,
                    prev_order_date,
                    prev_order_month,
                    cust_type,
                    CASE
                        WHEN order_date NOT NULL THEN 'Yes' ELSE 'No' END AS TM,
                    CASE 
                        WHEN prev_order_month = DATE(months, '-1 month') OR prev_order_month = months THEN 'Yes' ELSE 'No' END AS PM
                    FROM 
                    dates
                    ),

                    status AS
                    (
                    SELECT
                    customer_id,
                    months,
                    order_id,
                    order_date,
                    strftime('%Y', order_date) AS year,
                    reg_date,
                    reg_month,
                    prev_order_date,
                    prev_order_month,
                    cust_type,
                    TM,
                    PM,
                    CASE 
                        WHEN cust_type = 'Not Existing' THEN 'Not Existing'
                        WHEN cust_type = 'New' THEN 'New' 
                        WHEN cust_type = 'Existing' AND TM = 'No' THEN 'Non Active'
                        WHEN cust_type = 'Existing' AND TM = 'Yes' THEN 'Active'
                        ELSE 'Check' END AS cust_base,
                    CASE 
                        WHEN cust_type = 'Not Existing' THEN 'Not Existing'
                        WHEN cust_type = 'New' THEN 'New'
                        WHEN TM = 'Yes' AND PM = 'No' THEN 'Reactivated' 
                        WHEN TM = 'Yes' AND PM = 'Yes' THEN 'Retained'
                        WHEN TM = 'No' AND PM = 'No' THEN 'Dormant'
                        WHEN TM = 'No' AND PM = 'Yes' THEN 'Churned'
                        ELSE 'Check' END AS cust_status
                    FROM tmpm
                    )

                SELECT * FROM status 
                WHERE cust_status <> 'Dormant' AND cust_type <> 'Not Existing'
                ORDER BY months, order_date, order_id
                """
    
df_orders_cust_status = pd.read_sql(query_orders_cust_status, conn)
df_orders_cust_status.to_csv("orders_cust_status.csv", index=False)

In [42]:
conn.close()